In [1]:
import cv2
import numpy as np
import pickle
import time
import os

# --- PARAMETERS ---

# Calibration output from your provided script
CAMERA_CALIBRATION_FILE = 'output/calibration_data.pkl'
SAVED_ORIGIN_FILE = 'saved_origin.pkl'

SURFACE_ROWS, SURFACE_COLS = 5, 4
MARKER_LENGTH = 0.04       # meters (4cm)
GAP = 0.005                # meters (5mm gap)
SPACING = MARKER_LENGTH + GAP

SURFACE_MARKER_IDS = set(range(1, 21))  # IDs 1-20
ORIGIN_MARKER_IDS = (10, 11)            # For calibration
OBJECT_MARKER_ID = 0                    # Set to your object's marker ID

MIN_SURFACE_MARKERS = 6

CALIBRATION_TIME = 10                   # seconds to average the origin during startup

# --- ARUCO SETUP ---

aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_4X4_50)
aruco_params = cv2.aruco.DetectorParameters()
aruco_detector = cv2.aruco.ArucoDetector(aruco_dict, aruco_params)

# --- UTILITY FUNCTIONS ---

def marker_obj_points(marker_length):
    h = marker_length / 2
    return np.array(
        [[-h,  h, 0],
         [ h,  h, 0],
         [ h, -h, 0],
         [-h, -h, 0]], dtype=np.float32)

def rvec_tvec_to_mat(rvec, tvec):
    R, _ = cv2.Rodrigues(rvec)
    T = np.eye(4)
    T[:3, :3] = R
    T[:3, 3] = tvec.flatten()
    return T

# --- LOAD CALIBRATION ---

with open(CAMERA_CALIBRATION_FILE, 'rb') as f:
    calib_data = pickle.load(f)
camera_matrix = calib_data['camera_matrix']
dist_coeffs = calib_data['distortion_coefficients']

# --- CONSTRUCT SURFACE POINTS MODEL ---

surface_points = {}
for idx in range(20):
    row = idx // SURFACE_COLS
    col = idx % SURFACE_COLS
    # Center of marker
    surface_points[idx + 1] = np.array([col * SPACING, row * SPACING, 0.0])

# --- CAMERA CAPTURE ---
cap = cv2.VideoCapture(0)

# --- ORIGIN CALIBRATION ---

if not os.path.exists(SAVED_ORIGIN_FILE):
    print(f'Calibrating origin between markers {ORIGIN_MARKER_IDS}...')
    start_time = time.time()
    origin_samples = []
    while time.time() - start_time < CALIBRATION_TIME:
        ret, frame = cap.read()
        if not ret:
            continue
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        corners, ids, _ = aruco_detector.detectMarkers(gray)
        if ids is not None and len(ids) > 0:
            detected_ids = ids.flatten()
            if all(m_id in detected_ids for m_id in ORIGIN_MARKER_IDS):
                idx1 = np.where(detected_ids == ORIGIN_MARKER_IDS[0])[0][0]
                idx2 = np.where(detected_ids == ORIGIN_MARKER_IDS[1])[0][0]
                # Use surface_points model for (theoretical) centers
                origin_1 = surface_points[ORIGIN_MARKER_IDS[0]]
                origin_2 = surface_points[ORIGIN_MARKER_IDS[1]]
                midpoint = (origin_1 + origin_2) / 2
                origin_samples.append(midpoint)
    if origin_samples:
        new_origin = np.mean(origin_samples, axis=0)
        with open(SAVED_ORIGIN_FILE, 'wb') as f:
            pickle.dump(new_origin, f)
        print('Origin calibrated and saved.')
    else:
        print('Failed to calibrate origin. Try again!')
        cap.release()
        exit()
else:
    with open(SAVED_ORIGIN_FILE, 'rb') as f:
        new_origin = pickle.load(f)
    print(f'Loaded previously saved origin: {new_origin}')

# --- MAIN LOOP ---

while True:
    ret, frame = cap.read()
    if not ret:
        break
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = aruco_detector.detectMarkers(gray)

    if ids is not None and len(ids) > 0:
        frame = cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        detected_ids = ids.flatten()
        # Build correspondence: surface marker 3D (model) <-> detected 2D (image)
        surface_obj_points = []
        surface_img_points = []
        for i, marker_id in enumerate(detected_ids):
            if marker_id in surface_points:
                surface_obj_points.append(surface_points[marker_id])
                surface_img_points.append(corners[i][0].mean(axis=0))
        # Estimate surface pose if enough markers are visible
        if len(surface_obj_points) >= MIN_SURFACE_MARKERS:
            surface_obj_points = np.array(surface_obj_points, dtype=np.float32)
            surface_img_points = np.array(surface_img_points, dtype=np.float32)
            success, surface_rvec, surface_tvec = cv2.solvePnP(
                surface_obj_points, surface_img_points,
                camera_matrix, dist_coeffs,
                flags=cv2.SOLVEPNP_ITERATIVE
            )
            T_cam_surface = rvec_tvec_to_mat(surface_rvec, surface_tvec)
            T_surface_cam = np.linalg.inv(T_cam_surface)
            # Find and process the object marker
            obj_idx = np.where(detected_ids == OBJECT_MARKER_ID)[0]
            if len(obj_idx) > 0:
                img_pts = corners[obj_idx[0]][0]
                obj_pts = marker_obj_points(MARKER_LENGTH)
                success, obj_rvec, obj_tvec = cv2.solvePnP(
                    obj_pts, img_pts, camera_matrix, dist_coeffs,
                    flags=cv2.SOLVEPNP_ITERATIVE
                )
                T_cam_obj = rvec_tvec_to_mat(obj_rvec, obj_tvec)
                T_surface_obj = T_surface_cam @ T_cam_obj
                obj_pos_in_surface = T_surface_obj[:3, 3]
                obj_pos_relative = obj_pos_in_surface - new_origin
                # Show position on frame
                cv2.drawFrameAxes(frame, camera_matrix, dist_coeffs, obj_rvec, obj_tvec, MARKER_LENGTH * 2)
                cv2.putText(frame, f'Obj Pos: {obj_pos_relative.round(3)}',
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
            # Display surface axes (origin, X, Y, Z)
            axes_model = np.float32([[0,0,0], [0.05,0,0], [0,0.05,0], [0,0,0.05]])
            imgpts, _ = cv2.projectPoints(axes_model, surface_rvec, surface_tvec, camera_matrix, dist_coeffs)
            origin_ax = tuple(imgpts[0].ravel().astype(int))
            x_ax = tuple(imgpts[1].ravel().astype(int))
            y_ax = tuple(imgpts[2].ravel().astype(int))
            z_ax = tuple(imgpts[3].ravel().astype(int))
            cv2.line(frame, origin_ax, x_ax, (0,0,255), 3)
            cv2.line(frame, origin_ax, y_ax, (0,255,0), 3)
            cv2.line(frame, origin_ax, z_ax, (255,0,0), 3)
    cv2.imshow("Live Pose Estimation", frame)
    if cv2.waitKey(1) == 27:
        break
cap.release()
cv2.destroyAllWindows()


FileNotFoundError: [Errno 2] No such file or directory: 'output/calibration_data.pkl'